In [1]:
from demoparser2 import DemoParser

parser = DemoParser("cache/parsing_files/g2-vs-heroic-m1-ancient.dem")

event_df = parser.parse_event("player_death", player=["X", "Y"], other=["total_rounds_played"])
ticks_df = parser.parse_ticks(["X", "Y"])
event_df

,assistedflash,assister_X,assister_Y,assister_name,assister_steamid,attacker_X,attacker_Y,attacker_name,attacker_steamid,attackerblind,...,total_rounds_played,user_X,user_Y,user_name,user_steamid,weapon,weapon_fauxitemid,weapon_itemid,weapon_originalowner_xuid,wipe
0,False,NaN,NaN,None,None,-633.113770,-850.864929,TeSeS,76561197996678278,False,...,0,-539.514038,586.990967,huNter,76561198012872053,ak47,17293822569165553671,23868618176,,0
1,False,NaN,NaN,None,None,-1254.712769,-259.636169,m0NESY,76561198074762801,False,...,0,-722.428223,-203.876404,NertZ,76561198053126645,usp_silencer_off,17293822569123217469,39934640202,,0
2,False,NaN,NaN,None,None,-1234.539917,-135.652222,NiKo,76561198041683378,False,...,0,-701.395081,-257.692993,TeSeS,76561197996678278,usp_silencer,17293822569177546813,27770856656,,0
3,True,-526.353027,363.664093,huNter,76561198012872053,-1239.651489,-165.850525,NiKo,76561198041683378,False,...,0,-639.779419,-293.027832,degster,76561198799340122,usp_silencer,17293822569177546813,27770856656,,0
4,False,NaN,NaN,None,None,-443.413452,62.051792,huNter,76561198012872053,False,...,0,-712.070862,-728.184387,kyxsan,76561198057282432,usp_silencer,17293822569138094141,40681904471,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,False,-419.639771,-532.049988,sjuush,76561198160709585,-428.681885,-580.442261,NertZ,76561198053126645,False,...,18,-1025.140259,-687.389771,malbsMd,76561198080703143,m4a1_silencer,17293822569179840572,40740166582,,0
134,False,NaN,NaN,None,None,-1200.714478,-939.913452,m0NESY,76561198074762801,False,...,18,-643.902771,-534.226318,NertZ,76561198053126645,ak47,17293822569121185799,31980002385,,0
135,False,NaN,NaN,None,None,1383.727539,-809.653625,degster,76561198799340122,False,...,18,4.355045,-773.424072,NiKo,76561198041683378,awp,17293822569125249033,40364991209,,0
136,False,NaN,NaN,None,None,-1265.565308,-1139.508179,TeSeS,76561197996678278,False,...,18,-1791.906250,-967.516846,m0NESY,76561198074762801,ak47,17293822569121185799,31980002385,,0


In [2]:
from demoparser2 import DemoParser


event_names = [
    "begin_new_match", "round_start", "round_end", "round_mvp", 
    "player_death", "bomb_planted", "bomb_defused", "hostage_rescued", 
    "weapon_fire", "flashbang_detonate", "hegrenade_detonate", 
    "molotov_detonate", "smokegrenade_detonate", "player_hurt", 
    "player_blind"
]
for event in event_names:
    parser = DemoParser("cache/parsing_files/g2-vs-heroic-m1-ancient.dem")
    df = parser.parse_event(event)
    print(f"Event: {event}")
    df



Event: begin_new_match
Event: round_start
Event: round_end
Event: round_mvp
Event: player_death
Event: bomb_planted
Event: bomb_defused
Event: hostage_rescued
Event: weapon_fire
Event: flashbang_detonate
Event: hegrenade_detonate
Event: molotov_detonate
Event: smokegrenade_detonate
Event: player_hurt
Event: player_blind


In [3]:
# Inspect the player_death event
player_deaths = parser.parse_event("player_death")
print(player_deaths.columns)  # Check available column names


Index(['assistedflash', 'assister_name', 'assister_steamid', 'attacker_name',
       'attacker_steamid', 'attackerblind', 'attackerinair', 'distance',
       'dmg_armor', 'dmg_health', 'dominated', 'headshot', 'hitgroup',
       'noreplay', 'noscope', 'penetrated', 'revenge', 'thrusmoke', 'tick',
       'user_name', 'user_steamid', 'weapon', 'weapon_fauxitemid',
       'weapon_itemid', 'weapon_originalowner_xuid', 'wipe'],
      dtype='object')


In [4]:
# Parse player deaths
player_deaths = parser.parse_event("player_death")

# Create a DataFrame to store KAST metrics
player_kast = {}

# Iterate over each unique player in the match
for player in set(player_deaths['attacker_name']).union(player_deaths['user_name']):
    player_kast[player] = {
        "Kills": len(player_deaths[player_deaths['attacker_name'] == player]),
        "Deaths": len(player_deaths[player_deaths['user_name'] == player]),
        "Assists": len(player_deaths[player_deaths['assister_name'] == player]),
        "Survived": 0,  # Will be updated with round_end data
    }

# Parse round_end events to count rounds survived
rounds = parser.parse_event("round_end")
for round_no in rounds['round']:
    players_in_round = player_deaths[player_deaths['tick'] <= round_no]
    for player in player_kast.keys():
        if player not in players_in_round['user_name'].values:
            player_kast[player]["Survived"] += 1

# Calculate KAST percentage
for player, stats in player_kast.items():
    total_rounds = len(rounds)
    kast_score = stats["Kills"] + stats["Assists"] + stats["Survived"]
    player_kast[player]["KAST"] = (kast_score / total_rounds) * 100


In [5]:
from awpy import Demo
from awpy.stats import kast

# Parse the demo file
demo_path = "cache/parsing_files/g2-vs-heroic-m1-ancient.dem"
demo = Demo(demo_path)

# Calculate KAST statistics
kast_stats = kast(demo)

# Display the KAST statistics
kast_stats


,name,steamid,team_name,kast_rounds,n_rounds,kast
0,degster,76561198799340122,all,14,19,73.684211
1,NertZ,76561198053126645,all,14,19,73.684211
2,NiKo,76561198041683378,all,14,19,73.684211
3,Snax,76561197982141573,all,12,19,63.157895
4,TeSeS,76561197996678278,all,15,19,78.947368
5,huNter,76561198012872053,all,15,19,78.947368
6,kyxsan,76561198057282432,all,14,19,73.684211
7,m0NESY,76561198074762801,all,18,19,94.736842
8,malbsMd,76561198080703143,all,16,19,84.210526
9,sjuush,76561198160709585,all,15,19,78.947368


In [16]:
import os

# Replace this with the actual path to your parsing_files directory
 # For macOS/Linux
# or
parsing_files_path = r'C:\Users\awrm7\Desktop\E-alytics\project-awrm71\cache\parsing_files'  # For Windows

# Verify the path exists
if os.path.exists(parsing_files_path):
    print(f"Parsing files directory found: {parsing_files_path}")
else:
    print(f"Parsing files directory not found: {parsing_files_path}")

import os
import glob

# List all .dem files in the parsing_files directory
demo_files = glob.glob(os.path.join(parsing_files_path, '*.dem'))

if demo_files:
    print("Demo files found:")
    for file in demo_files:
        print(file)
else:
    print("No demo files found in the specified parsing_files directory.")

Parsing files directory found: C:\Users\awrm7\Desktop\E-alytics\project-awrm71\cache\parsing_files
Demo files found:
C:\Users\awrm7\Desktop\E-alytics\project-awrm71\cache\parsing_files\g2-vs-heroic-m1-ancient.dem
C:\Users\awrm7\Desktop\E-alytics\project-awrm71\cache\parsing_files\g2-vs-heroic-m2-nuke.dem


In [33]:
import awpy
from awpy import Demo
from awpy.stats import kast

# Parse the demo file
demo_path = r'C:\Users\awrm7\Desktop\E-alytics\project-awrm71\cache\parsing_files\g2-vs-heroic-m2-nuke.dem'  # For Windows

demo = Demo(demo_path)

# Function to display selected columns of a dataframe
def display_selected_columns(df, columns):
    if df is not None:
        selected_df = df[columns]
        print(selected_df.head())  # Display the first few rows for testing
    else:
        print("DataFrame is empty or not loaded.")

# Display Kills DataFrame
kills_columns = [
    'tick', 'attacker_name', 'victim_name', 'weapon', 'headshot',
    'penetrated', 'thrusmoke', 'attacker_team_clan_name', 'victim_team_clan_name'
]
display_selected_columns(demo.kills, kills_columns)

# Display Damages DataFrame
damages_columns = [
    'tick', 'attacker_name', 'victim_name', 'weapon', 'dmg_health',
    'dmg_armor', 'attacker_team_clan_name', 'victim_team_clan_name'
]
display_selected_columns(demo.damages, damages_columns)

# Display Bomb Events DataFrame
bomb_columns = ['tick', 'event', 'site', 'X', 'Y', 'Z', 'round']
display_selected_columns(demo.bomb, bomb_columns)

# Display Grenades DataFrame
grenades_columns = [
    'tick', 'grenade_type', 'thrower',  'X', 'Y', 'Z', 'round'
]

display_selected_columns(demo.grenades, grenades_columns)

# Display Smokes DataFrame
smokes_columns = [
    'start_tick', 'end_tick', 'thrower_name', 'thrower_team_clan_name',
    'X', 'Y', 'Z', 'round'
]
display_selected_columns(demo.smokes, smokes_columns)

# Display Infernos DataFrame
infernos_columns = [
    'start_tick', 'end_tick', 'thrower_name', 'thrower_team_clan_name',
    'X', 'Y', 'Z', 'round'
]
display_selected_columns(demo.infernos, infernos_columns)


   tick attacker_name victim_name        weapon  headshot  penetrated  \
0  4190         TeSeS        NiKo  usp_silencer      True           0   
1  6447         TeSeS      huNter  usp_silencer      True           0   
2  6668         TeSeS      m0NESY  usp_silencer      True           0   
3  6795          Snax       TeSeS         glock      True           0   
4  7169         NertZ        Snax  usp_silencer      True           0   

   thrusmoke attacker_team_clan_name victim_team_clan_name  
0      False                  HEROIC            G2 Esports  
1      False                  HEROIC            G2 Esports  
2      False                  HEROIC            G2 Esports  
3      False              G2 Esports                HEROIC  
4      False                  HEROIC            G2 Esports  
   tick attacker_name victim_name        weapon hitgroup  dmg_health  \
0  4177          NiKo       TeSeS         glock     None          10   
1  4186          NiKo       TeSeS         glock    

In [ ]:
elif data_view == "Game Events":
            # Game Events Section
            st.write("### Game Events Viewer")

            # Dropdown to select event type
            event_type = st.sidebar.selectbox(
                "Select Event Type", ["kills", "damages", "bomb", "grenades", "smokes", "infernos"]
            )

            # Combine data from multiple matches
            combined_data = pd.DataFrame()
            for file_name, parsed_data in st.session_state["parsed_matches"].items():
                demo_path = f"temp_{file_name}"
                try:
                    event_data = get_game_events(demo_path, event_type)
                    combined_data = pd.concat([combined_data, event_data], ignore_index=True)
                except Exception as e:
                    st.warning(f"Failed to load {event_type} data for {file_name}: {e}")

            # If no data, show warning
            if combined_data.empty:
                st.warning(f"No {event_type} data available.")
            else:
                # Let users select columns to display
                available_columns = list(combined_data.columns)
                selected_columns = st.sidebar.multiselect("Select Columns", available_columns, default=available_columns)

                # Display filtered data
                st.write(f"### {event_type.capitalize()} Data")
                st.dataframe(combined_data[selected_columns])

                # Download button
                st.download_button(
                    label="Download Data as CSV",
                    data=combined_data[selected_columns].to_csv(index=False).encode("utf-8"),
                    file_name=f"{event_type}_data.csv",
                    mime="text/csv",
                )

In [31]:
print(demo.damages.columns)


Index(['tick', 'weapon', 'dmg_armor', 'dmg_health', 'hitgroup',
       'is_bomb_planted', 'attacker_X', 'attacker_Y', 'attacker_Z',
       'attacker_pitch', 'attacker_yaw', 'attacker_last_place_name',
       'attacker_health', 'attacker_armor_value',
       'attacker_current_equip_value', 'attacker_has_defuser',
       'attacker_has_helmet', 'attacker_inventory', 'attacker_ping',
       'attacker_team_name', 'attacker_team_clan_name', 'attacker_name',
       'attacker_steamid', 'victim_X', 'victim_Y', 'victim_Z', 'victim_pitch',
       'victim_yaw', 'victim_last_place_name', 'victim_health',
       'victim_armor_value', 'victim_current_equip_value',
       'victim_has_defuser', 'victim_has_helmet', 'victim_inventory',
       'victim_ping', 'victim_team_name', 'victim_team_clan_name',
       'victim_name', 'dmg_health_real', 'round', 'ticks_since_round_start',
       'ticks_since_freeze_time_end', 'ticks_since_bomb_plant', 'clock'],
      dtype='object')


In [7]:
import pandas as pd
from awpy import Demo

# Parse the demo file


# Load the kills DataFrame
kills_df = demo.kills

# Calculate total stats for each player (kills, assists, deaths) across both sides
player_stats = (
    kills_df.groupby(["attacker_name", "attacker_team_name", "attacker_team_clan_name"])
    .agg(
        kills=("attacker_name", "count"),
        assists=("assister_name", "count"),
    )
    .reset_index()
    .rename(
        columns={
            "attacker_name": "player_name",
            "attacker_team_name": "team_name",
            "attacker_team_clan_name": "clan_name",
        }
    )
)

# Calculate total deaths for each player
deaths_stats = (
    kills_df.groupby(["victim_name", "victim_team_name", "victim_team_clan_name"])
    .agg(deaths=("victim_name", "count"))
    .reset_index()
    .rename(
        columns={
            "victim_name": "player_name",
            "victim_team_name": "team_name",
            "victim_team_clan_name": "clan_name",
        }
    )
)

# Merge kills/assists with deaths on player_name, team_name, and clan_name
final_stats = pd.merge(
    player_stats, deaths_stats, on=["player_name", "team_name", "clan_name"], how="outer"
).fillna(0)

# Ensure correct data types for numeric columns
final_stats["kills"] = final_stats["kills"].astype(int)
final_stats["assists"] = final_stats["assists"].astype(int)
final_stats["deaths"] = final_stats["deaths"].astype(int)

# Add a "Both" row by summing stats across both sides for each player
both_stats = (
    final_stats.groupby(["player_name", "clan_name"])
    .agg(
        kills=("kills", "sum"),
        assists=("assists", "sum"),
        deaths=("deaths", "sum"),
    )
    .reset_index()
    .assign(team_name="Both")  # Assign "Both" as the team_name for these rows
)

# Combine the original stats with the "Both" rows
combined_stats = pd.concat([final_stats, both_stats], ignore_index=True)

# Sort the data for better readability
combined_stats = combined_stats.sort_values(by=["team_name", "player_name"]).reset_index(drop=True)

# Display the resulting DataFrame
combined_stats


,player_name,team_name,clan_name,kills,assists,deaths
0,degster,Both,HEROIC,18,6,10
1,NertZ,Both,HEROIC,16,6,14
2,NiKo,Both,G2 Esports,22,8,12
3,Snax,Both,G2 Esports,7,3,14
4,TeSeS,Both,HEROIC,13,2,13
5,huNter,Both,G2 Esports,8,1,14
6,kyxsan,Both,HEROIC,10,5,15
7,m0NESY,Both,G2 Esports,18,4,13
8,malbsMd,Both,G2 Esports,13,7,14
9,sjuush,Both,HEROIC,8,4,15


In [8]:
import pandas as pd
from awpy import Demo
from awpy.stats import kast

# Parse the demo file

# Load the kills DataFrame
kills_df = demo.kills

# Calculate total stats for each player (kills, assists, deaths) across both sides
player_stats = (
    kills_df.groupby(["attacker_name", "attacker_team_name", "attacker_team_clan_name"])
    .agg(
        kills=("attacker_name", "count"),
        assists=("assister_name", "count"),
    )
    .reset_index()
    .rename(
        columns={
            "attacker_name": "player_name",
            "attacker_team_name": "team_name",
            "attacker_team_clan_name": "clan_name",
        }
    )
)

# Calculate total deaths for each player
deaths_stats = (
    kills_df.groupby(["victim_name", "victim_team_name", "victim_team_clan_name"])
    .agg(deaths=("victim_name", "count"))
    .reset_index()
    .rename(
        columns={
            "victim_name": "player_name",
            "victim_team_name": "team_name",
            "victim_team_clan_name": "clan_name",
        }
    )
)

# Merge kills/assists with deaths on player_name, team_name, and clan_name
final_stats = pd.merge(
    player_stats, deaths_stats, on=["player_name", "team_name", "clan_name"], how="outer"
).fillna(0)

# Ensure correct data types for numeric columns
final_stats["kills"] = final_stats["kills"].astype(int)
final_stats["assists"] = final_stats["assists"].astype(int)
final_stats["deaths"] = final_stats["deaths"].astype(int)

# Add a "Both" row by summing stats across both sides for each player
both_stats = (
    final_stats.groupby(["player_name", "clan_name"])
    .agg(
        kills=("kills", "sum"),
        assists=("assists", "sum"),
        deaths=("deaths", "sum"),
    )
    .reset_index()
    .assign(team_name="Both")  # Assign "Both" as the team_name for these rows
)

# Combine the original stats with the "Both" rows
combined_stats = pd.concat([final_stats, both_stats], ignore_index=True)

# Sort the data for better readability
combined_stats = combined_stats.sort_values(by=["team_name", "player_name"]).reset_index(drop=True)

# Calculate KAST statistics using awpy's kast function
kast_stats = kast(demo)

# Process the KAST DataFrame to make it compatible for merging
kast_df = (
    pd.DataFrame(kast_stats)
    .rename(columns={"name": "player_name", "team_name": "team_name", "kast": "kast_percentage"})
    .drop(columns=["steamid"])  # Remove steamid as it's not required
)

# Normalize the "all" side to "Both" for consistency
kast_df["team_name"] = kast_df["team_name"].replace("all", "Both")

# Merge the KAST stats into the combined_stats DataFrame
final_combined_stats = pd.merge(
    combined_stats,
    kast_df,
    on=["player_name", "team_name"],
    how="left"
).fillna(0)

# Sort the final DataFrame for better readability
final_combined_stats = final_combined_stats.sort_values(by=["team_name", "player_name"]).reset_index(drop=True)

# Display the resulting DataFrame
from awpy.stats import adr

# Calculate ADR statistics using awpy's adr function
adr_stats = adr(demo)

# Process the ADR DataFrame to make it compatible for merging
adr_df = (
    pd.DataFrame(adr_stats)
    .rename(columns={"name": "player_name", "team_name": "team_name", "dmg": "total_damage", "adr": "average_damage_per_round"})
    .drop(columns=["steamid", "n_rounds"])  # Remove unnecessary columns
)

# Normalize the "all" side to "Both" for consistency
adr_df["team_name"] = adr_df["team_name"].replace("all", "Both")

# Merge the ADR stats into the existing combined stats DataFrame
final_combined_stats = pd.merge(
    final_combined_stats,
    adr_df,
    on=["player_name", "team_name"],
    how="left"
).fillna(0)

# Sort the final DataFrame for better readability
final_combined_stats = final_combined_stats.sort_values(by=["team_name", "player_name"]).reset_index(drop=True)

# Display the resulting DataFrame
final_combined_stats



,player_name,team_name,clan_name,kills,assists,deaths,kast_rounds,n_rounds,kast_percentage,total_damage,average_damage_per_round
0,degster,Both,HEROIC,18,6,10,14,19,73.684211,1579,83.105263
1,NertZ,Both,HEROIC,16,6,14,14,19,73.684211,1995,105.000000
2,NiKo,Both,G2 Esports,22,8,12,14,19,73.684211,2139,112.578947
3,Snax,Both,G2 Esports,7,3,14,12,19,63.157895,978,51.473684
4,TeSeS,Both,HEROIC,13,2,13,15,19,78.947368,1508,79.368421
5,huNter,Both,G2 Esports,8,1,14,15,19,78.947368,1144,60.210526
6,kyxsan,Both,HEROIC,10,5,15,14,19,73.684211,1251,65.842105
7,m0NESY,Both,G2 Esports,18,4,13,18,19,94.736842,1873,98.578947
8,malbsMd,Both,G2 Esports,13,7,14,16,19,84.210526,1190,62.631579
9,sjuush,Both,HEROIC,8,4,15,15,19,78.947368,1028,54.105263


In [9]:
import pandas as pd
from awpy import Demo
from awpy.stats import kast, adr, rating

# Parse the demo file

# Load the kills DataFrame
kills_df = demo.kills

# Separate data for valid kills (no team kills or suicides)
valid_kills_df = kills_df[
    (kills_df["attacker_team_clan_name"] != kills_df["victim_team_clan_name"]) &  # Exclude team kills
    (kills_df["attacker_name"] != kills_df["victim_name"])                        # Exclude suicides
]

# Calculate total stats for valid kills (kills and assists)
player_stats = (
    valid_kills_df.groupby(["attacker_name", "attacker_team_name", "attacker_team_clan_name"])
    .agg(
        kills=("attacker_name", "count"),
        assists=("assister_name", "count"),
    )
    .reset_index()
    .rename(
        columns={
            "attacker_name": "player_name",
            "attacker_team_name": "team_name",
            "attacker_team_clan_name": "clan_name",
        }
    )
)

# Calculate total deaths for all victims (including team kills and suicides)
deaths_stats = (
    kills_df.groupby(["victim_name", "victim_team_name", "victim_team_clan_name"])
    .agg(deaths=("victim_name", "count"))
    .reset_index()
    .rename(
        columns={
            "victim_name": "player_name",
            "victim_team_name": "team_name",
            "victim_team_clan_name": "clan_name",
        }
    )
)

# Merge valid kills/assists with deaths on player_name, team_name, and clan_name
final_stats = pd.merge(
    player_stats, deaths_stats, on=["player_name", "team_name", "clan_name"], how="outer"
).fillna(0)

# Ensure correct data types for numeric columns
final_stats["kills"] = final_stats["kills"].astype(int)
final_stats["assists"] = final_stats["assists"].astype(int)
final_stats["deaths"] = final_stats["deaths"].astype(int)

# Add a "Both" row by summing stats across both sides for each player
both_stats = (
    final_stats.groupby(["player_name", "clan_name"])
    .agg(
        kills=("kills", "sum"),
        assists=("assists", "sum"),
        deaths=("deaths", "sum"),
    )
    .reset_index()
    .assign(team_name="Both")  # Assign "Both" as the team_name for these rows
)

# Combine the original stats with the "Both" rows
combined_stats = pd.concat([final_stats, both_stats], ignore_index=True)

# Sort the data for better readability
combined_stats = combined_stats.sort_values(by=["team_name", "player_name"]).reset_index(drop=True)

# Calculate KAST statistics
kast_stats = kast(demo)
kast_df = (
    pd.DataFrame(kast_stats)
    .rename(columns={"name": "player_name", "team_name": "team_name", "kast": "kast_percentage"})
    .drop(columns=["steamid"])
)
kast_df["team_name"] = kast_df["team_name"].replace("all", "Both")  # Normalize "all" to "Both"

# Merge KAST stats
combined_stats = pd.merge(
    combined_stats,
    kast_df,
    on=["player_name", "team_name"],
    how="left"
).fillna(0)

# Calculate ADR statistics
adr_stats = adr(demo)
adr_df = (
    pd.DataFrame(adr_stats)
    .rename(columns={"name": "player_name", "team_name": "team_name", "dmg": "total_damage", "adr": "average_damage_per_round"})
    .drop(columns=["steamid", "n_rounds"])
)
adr_df["team_name"] = adr_df["team_name"].replace("all", "Both")  # Normalize "all" to "Both"

# Merge ADR stats
combined_stats = pd.merge(
    combined_stats,
    adr_df,
    on=["player_name", "team_name"],
    how="left"
).fillna(0)

# Calculate Rating 2.0 and Impact statistics
rating_stats = rating(demo)
rating_df = (
    pd.DataFrame(rating_stats)
    .rename(columns={"name": "player_name", "team_name": "team_name", "rating": "rating_2.0", "impact": "impact"})
    .drop(columns=["steamid", "n_rounds"])
)
rating_df["team_name"] = rating_df["team_name"].replace("all", "Both")  # Normalize "all" to "Both"

# Merge Rating stats
combined_stats = pd.merge(
    combined_stats,
    rating_df,
    on=["player_name", "team_name"],
    how="left"
).fillna(0)

# Sort the final DataFrame for better readability
combined_stats = combined_stats.sort_values(by=["team_name", "player_name"]).reset_index(drop=True)

# Display the resulting DataFrame
combined_stats


,player_name,team_name,clan_name,kills,assists,deaths,kast_rounds,n_rounds,kast_percentage,total_damage,average_damage_per_round,impact,rating_2.0
0,degster,Both,HEROIC,18,6,10,14,19,73.684211,1579,83.105263,1.652105,1.414137
1,NertZ,Both,HEROIC,16,6,14,14,19,73.684211,1995,105.000000,1.538421,1.307245
2,NiKo,Both,G2 Esports,22,8,12,14,19,73.684211,2139,112.578947,2.144737,1.644811
3,Snax,Both,G2 Esports,7,3,14,12,19,63.157895,978,51.473684,0.507368,0.644453
4,TeSeS,Both,HEROIC,13,2,13,15,19,78.947368,1508,79.368421,1.135789,1.139488
5,huNter,Both,G2 Esports,8,1,14,15,19,78.947368,1144,60.210526,0.575263,0.822679
6,kyxsan,Both,HEROIC,10,5,15,14,19,73.684211,1251,65.842105,0.843684,0.875701
7,m0NESY,Both,G2 Esports,16,2,13,18,19,94.736842,1873,98.578947,1.674211,1.538439
8,malbsMd,Both,G2 Esports,13,7,14,16,19,84.210526,1190,62.631579,1.091579,1.085817
9,sjuush,Both,HEROIC,8,4,15,15,19,78.947368,1028,54.105263,0.663684,0.796068


In [10]:
import plotly.io as pio
pio.renderers.default = "notebook"  # Use "notebook" renderer for Jupyter Notebooks


In [ ]:
import pandas as pd
import streamlit as st
import plotly.express as px

# Placeholder data for combined_stats
data = {
    "player_name": ["Player1", "Player2", "Player3", "Player4", "Player5"],
    "team_name": ["CT", "CT", "TERRORIST", "TERRORIST", "Both"],
    "clan_name": ["Team A", "Team A", "Team B", "Team B", "Team A"],
    "kills": [10, 15, 20, 5, 25],
    "assists": [5, 7, 3, 2, 8],
    "deaths": [8, 10, 12, 4, 20],
    "kast_percentage": [70.5, 80.2, 60.1, 55.3, 85.0],
    "total_damage": [1500, 1800, 1700, 1200, 2000],
    "average_damage_per_round": [75.0, 90.0, 85.0, 60.0, 100.0],
    "impact": [1.1, 1.3, 1.0, 0.8, 1.5],
    "rating_2.0": [1.2, 1.4, 1.1, 0.9, 1.6],
}

combined_stats = pd.DataFrame(data)

# Extended placeholder data for kills_df
kills_data = {
    "attacker_name": [
        "Player1", "Player1", "Player1", "Player1", "Player1", "Player1",
        "Player2", "Player2", "Player3", "Player4", "Player5", "Player5",
    ],
    "victim_name": [
        "Player3", "Player4", "Player3", "Player4", "Player5", "Player5",
        "Player5", "Player1", "Player1", "Player2", "Player3", "Player4",
    ],
    "weapon": [
        "AK-47", "M4A1", "AWP", "AK-47", "P250", "Deagle",
        "AWP", "AK-47", "P90", "P250", "Deagle", "AK-47",
    ],
    "dmg_health": [
        90, 100, 150, 70, 50, 120,
        130, 120, 140, 50, 80, 100,
    ],
    "headshot": [
        True, False, True, False, False, True,
        True, True, False, False, True, True,
    ],
    "attacker_team_clan_name": [
        "Team A", "Team A", "Team A", "Team A", "Team A", "Team A",
        "Team A", "Team A", "Team B", "Team B", "Team A", "Team A",
    ],
    "victim_team_clan_name": [
        "Team B", "Team B", "Team B", "Team B", "Team B", "Team B",
        "Team A", "Team B", "Team A", "Team A", "Team B", "Team B",
    ],
}
kills_df = pd.DataFrame(kills_data)

# Function to generate kill details with bold labels for kills
def generate_kill_details(group):
    details = []
    for i, row in enumerate(group.itertuples(), 1):
        details.append(
            f"<b>Kill {i}:</b> Weapon: {row.weapon}, Damage: {row.dmg_health}, Headshot: {row.headshot}"
        )
    return "<br>".join(details)

# Filter out team kills or self-kills
filtered_kills_df = kills_df[
    kills_df["attacker_team_clan_name"] != kills_df["victim_team_clan_name"]
]

# Process data for heatmaps
def process_team_data(team_name):
    team_kills = filtered_kills_df[filtered_kills_df["attacker_team_clan_name"] == team_name]
    head_to_head = (
        team_kills.groupby(["attacker_name", "victim_name"])
        .agg(
            kills=("attacker_name", "count"),
            details=("weapon", lambda x: generate_kill_details(team_kills.loc[x.index])),
        )
        .reset_index()
    )
    heatmap_data = head_to_head.pivot(index="attacker_name", columns="victim_name", values="kills").fillna(0)
    hover_data = head_to_head.pivot(index="attacker_name", columns="victim_name", values="details").fillna("")
    return heatmap_data, hover_data

# Generate heatmap data for each team
team1_heatmap_data, team1_hover_data = process_team_data("Team A")
team2_heatmap_data, team2_hover_data = process_team_data("Team B")

# Streamlit App
def main():
    st.title("Interactive Heatmap Example")
    
    st.header("Team A Heatmap")
    fig1 = px.imshow(
        team1_heatmap_data,
        labels=dict(x="Victim", y="Attacker", color="Kills"),
        title="Team A Head-to-Head Kills",
        text_auto=True,
        color_continuous_scale="Blues",
    )
    fig1.update_traces(
        hovertemplate="<b>Attacker:</b> %{y}<br><b>Victim:</b> %{x}<br><b>Kills:</b> %{z}<br><b>Details:</b><br>%{customdata}",
        customdata=team1_hover_data.values,
    )
    st.plotly_chart(fig1)

    st.header("Team B Heatmap")
    fig2 = px.imshow(
        team2_heatmap_data,
        labels=dict(x="Victim", y="Attacker", color="Kills"),
        title="Team B Head-to-Head Kills",
        text_auto=True,
        color_continuous_scale="Reds",
    )
    fig2.update_traces(
        hovertemplate="<b>Attacker:</b> %{y}<br><b>Victim:</b> %{x}<br><b>Kills:</b> %{z}<br><b>Details:</b><br>%{customdata}",
        customdata=team2_hover_data.values,
    )
    st.plotly_chart(fig2)

if __name__ == "__main__":
    main()


Attacker: TeSeS<br>Victim: NiKo<br><b>Kill 1:</b><br>- Weapon: ak47<br>- Damage: 108<br>- Headshot: True<br><b>Kill 2:</b><br>- Weapon: m4a1<br>- Damage: 126<br>- Headshot: True<br><b>Kill 3:</b><br>- Weapon: ak47<br>- Damage: 34<br>- Headshot: False<br><b>Kill 4:</b><br>- Weapon: usp_silencer<br>- Damage: 118<br>- Headshot: True<br><b>Kill 5:</b><br>- Weapon: mp9<br>- Damage: 13<br>- Headshot: False<br><b>Kill 6:</b><br>- Weapon: mp9<br>- Damage: 15<br>- Headshot: False<br><b>Kill 7:</b><br>- Weapon: ak47<br>- Damage: 109<br>- Headshot: True


In [38]:
def extract_map_name_from_filename(file_name):
    """
    Extract the map name from the file name.
    Assumes the map name is part of the file name, e.g., 'g2-vs-heroic-m1-ancient.dem'.

    Parameters:
        file_name (str): The name of the demo file.

    Returns:
        str: The extracted map name with 'de_' prefix for compatibility.
    """
    return f"de_{file_name.split('-')[-1].replace('.dem', '')}"

# Test the function with a sample file name
file_name = "g2-vs-heroic-m1-ancient.dem"
map_name = extract_map_name_from_filename(file_name)
map_name


'de_ancient'

In [35]:
from awpy.data.map_data  import MAP_DATA

# Print all available maps
print(MAP_DATA.keys())


dict_keys(['ar_baggage', 'ar_shoots', 'cs_office', 'cs_italy', 'de_ancient', 'de_anubis', 'de_dust', 'de_dust2', 'de_inferno', 'de_inferno_s2', 'de_mirage', 'de_nuke', 'de_overpass', 'de_vertigo'])


In [41]:
kills_df.columns

Index(['tick', 'assistedflash', 'dmg_health', 'dmg_armor', 'attackerblind',
       'headshot', 'hitgroup', 'noscope', 'penetrated', 'thrusmoke',
       'is_bomb_planted', 'weapon', 'assister_X', 'assister_Y', 'assister_Z',
       'assister_pitch', 'assister_yaw', 'assister_last_place_name',
       'assister_flash_duration', 'assister_health', 'assister_armor_value',
       'assister_current_equip_value', 'assister_has_defuser',
       'assister_has_helmet', 'assister_inventory', 'assister_ping',
       'assister_team_name', 'assister_team_clan_name', 'assister_name',
       'assister_steamid', 'attacker_X', 'attacker_Y', 'attacker_Z',
       'attacker_pitch', 'attacker_yaw', 'attacker_last_place_name',
       'attacker_flash_duration', 'attacker_health', 'attacker_armor_value',
       'attacker_current_equip_value', 'attacker_has_defuser',
       'attacker_has_helmet', 'attacker_inventory', 'attacker_ping',
       'attacker_team_name', 'attacker_team_clan_name', 'attacker_name',
      

In [43]:
def validate_kills_df(kills_df, map_name):
    """
    Validate the kills_df for required columns and ensure data exists for the map.

    Parameters:
        kills_df (DataFrame): The kills dataframe.
        map_name (str): The map name to filter.

    Returns:
        DataFrame: Filtered dataframe for the given map.
    """
    required_columns = [
        "attacker_X",
        "attacker_y",
        "victim_pos_x",
        "victim_pos_y",
        "map_name",
    ]
    for col in required_columns:
        if col not in kills_df.columns:
            raise ValueError(f"Missing required column: {col} in kills_df.")

    filtered_df = kills_df[kills_df["map_name"] == map_name]
    if filtered_df.empty:
        raise ValueError(f"No kills data available for map: {map_name}.")

    return filtered_df

# Test the function with placeholder data
valid_kills_df = validate_kills_df(kills_df, map_name)
valid_kills_df.head()

ValueError: Missing required column: attacker_y in kills_df.